# Lab | Cross Validation

For this lab, we will build a model on customer churn binary classification problem. You will be using `files_for_lab/Customer-Churn.csv` file.

### Instructions

1. Apply SMOTE for upsampling the data

    - Use logistic regression to fit the model and compute the accuracy of the model.
    - Use decision tree classifier to fit the model and compute the accuracy of the model.
    - Compare the accuracies of the two models.

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks

In [20]:
path = 'files_for_lab/Customer-Churn.csv'
data = pd.read_csv(path)
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [21]:
# Check null values
data.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [22]:
# Check data types
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [23]:
# Change total charges to numerical data
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors="coerce")

In [24]:
# Check if the shift to numerical data created any NaN values
data['TotalCharges'].isna().sum()

11

In [25]:
# Check the amount of NaNs relative to the dataset:
percent = data['TotalCharges'].isna().sum() * 100 / data.shape[0]
print(percent)

0.15618344455487718


In [26]:
# As the rows represent 0.15% of the dataset, we could remove them:
data.dropna(inplace=True)

In [27]:
# X - y split
y = data['Churn']
X = data.drop(y.name, axis=1)

In [28]:
# Num - cat split
num = X.select_dtypes(np.number)
cat = X.select_dtypes(object)

In [29]:
# Encode categoricals
cat = pd.get_dummies(cat, columns=cat.columns, drop_first=True)

In [30]:
# Reconnect num & cat columns
X = pd.concat([num, cat], axis=1)

In [31]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, \
    random_state=42)

In [32]:
# Apply SMOTE oversampling to the test data
X_sm, y_sm = SMOTE().fit_resample(X_test, y_test)

In [33]:
# Fit logistic regression model to data
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_sm)
confusion_matrix(y_pred, y_sm)

array([[1371,  615],
       [ 178,  934]])

In [34]:
# Fit Decision Tree model to data
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_sm)
confusion_matrix(y_pred, y_sm)

array([[1235,  760],
       [ 314,  789]])

2. Apply TomekLinks for downsampling

    - It is important to remember that it does not make the two classes equal but only removes the points from the majority class that are close to other points in minority class.
    - Use logistic regression to fit the model and compute the accuracy of the model.
    - Use decision tree classifier to fit the model and compute the accuracy of the model.
    - Compare the accuracies of the two models.
    - You can also apply this algorithm one more time and check the how the imbalance in the two classes changed from the last time.

In [35]:
# Apply Tomek-Links undersampling to the test data
X_tl, y_tl = TomekLinks().fit_resample(X_test, y_test)

In [36]:
# Fit logistic regression model to data
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_tl)
confusion_matrix(y_pred, y_tl)

array([[1258,  273],
       [ 122,  288]])

In [37]:
# Fit Decision Tree model to data
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_tl)
confusion_matrix(y_pred, y_tl)

array([[1122,  277],
       [ 258,  284]])